In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_fscore_support, balanced_accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import Flower related libraries
import flwr as fl
from typing import Dict, List, Optional, Tuple, Union
from flwr.common import NDArrays, Scalar, Parameters, FitIns, EvaluateIns, FitRes, EvaluateRes, parameters_to_ndarrays, ndarrays_to_parameters
import pickle
import json
import os

# Load and preprocess the dataset
def load_and_preprocess_data(file_path='./transactions_data_extended.csv'):
    df = pd.read_csv(file_path)

    # Parse mixed date formats
    def parse_date(date_str):
        for fmt in ["%Y-%m-%d %H:%M:%S", "%d-%m-%Y", "%Y-%m-%d"]:
            try:
                return datetime.strptime(date_str.split()[0], fmt)
            except ValueError:
                continue
        return pd.NaT

    df['date'] = df['date'].apply(parse_date)
    df = df.dropna(subset=['date'])

    # Extract date features
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day_of_week'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    df['is_month_start'] = df['day'].apply(lambda x: 1 if x <= 5 else 0)
    df['is_month_end'] = df['day'].apply(lambda x: 1 if x >= 25 else 0)

    # Amount-based features
    if 'amount' in df.columns:
        df['amount_log'] = np.log1p(df['amount'].abs())
        df['is_large_amount'] = df['amount'].apply(lambda x: 1 if x > df['amount'].quantile(0.75) else 0)
        df['is_very_large_amount'] = df['amount'].apply(lambda x: 1 if x > df['amount'].quantile(0.95) else 0)

    # Encode categorical features
    label_encoders = {}
    for col in ['description', 'category', 'clean_description']:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le

    # Normalize numerical features
    numerical_features = ['amount', 'amount_log', 'day', 'month', 'year', 'day_of_week', 'quarter', 'week_of_year']
    numerical_features = [f for f in numerical_features if f in df.columns]
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    print(f"Dataset shape: {df.shape}")
    if 'category' in df.columns:
        print(f"Number of categories: {df['category'].nunique()}")

    return df, label_encoders

# Prepare datasets for each model
def prepare_model_datasets(df):
    # Common features
    base_features = ['day', 'month', 'year', 'day_of_week', 'is_weekend', 'is_month_start', 
                    'is_month_end', 'quarter', 'week_of_year']
    amount_features = ['amount', 'amount_log', 'is_large_amount']

    # 1. Fraud Detection
    fraud_df = df.copy()
    fraud_df['is_fraud'] = fraud_df.get('is_fraud', fraud_df['is_very_large_amount'])
    fraud_features = amount_features + ['is_very_large_amount'] + base_features
    if 'category' in df.columns:
        fraud_features.append('category')
    if 'clean_description' in df.columns:
        fraud_features.append('clean_description')
    fraud_features = [f for f in fraud_features if f in df.columns]
    X_fraud, y_fraud = fraud_df[fraud_features], fraud_df['is_fraud']

    # 2. Budget Alerts
    budget_df = df[df['is_fraud'] == 0] if 'is_fraud' in df.columns else df.copy()
    budget_target = 'category'
    if 'category' in budget_df.columns and budget_df['category'].nunique() > 50:
        top_categories = budget_df['category'].value_counts().nlargest(10).index
        budget_df['category_simplified'] = budget_df['category'].apply(lambda x: x if x in top_categories else -1)
        budget_target = 'category_simplified'
    budget_features = amount_features + base_features
    budget_features = [f for f in budget_features if f in budget_df.columns]
    X_budget, y_budget = budget_df[budget_features], budget_df[budget_target]

    # 3. Expense Tracking
    expense_df = budget_df.copy()
    expense_features = base_features
    if 'category' in expense_df.columns:
        expense_features.append('category')
    expense_features = [f for f in expense_features if f in expense_df.columns]
    X_expense = expense_df[expense_features]
    y_expense = expense_df['amount_log'] if 'amount_log' in expense_df.columns else expense_df['amount']

    # 4. Financial Forecasting
    forecast_df = df.sort_values('date').copy()
    if 'amount' in forecast_df.columns:
        for window in [3, 7, 14, 30]:
            forecast_df[f'rolling_avg_{window}'] = forecast_df['amount'].rolling(window=window).mean().fillna(0)
            forecast_df[f'rolling_std_{window}'] = forecast_df['amount'].rolling(window=window).std().fillna(0)
        forecast_df['amount_diff_1'] = forecast_df['amount'].diff(1).fillna(0)
    forecast_features = base_features + [col for col in forecast_df.columns if 'rolling_' in col or 'amount_diff' in col]
    forecast_features = [f for f in forecast_features if f in forecast_df.columns]
    X_forecast = forecast_df[forecast_features]
    y_forecast = forecast_df['amount_log'] if 'amount_log' in forecast_df.columns else forecast_df['amount']

    return {
        'fraud': (X_fraud, y_fraud),
        'budget': (X_budget, y_budget),
        'expense': (X_expense, y_expense),
        'forecast': (X_forecast, y_forecast)
    }

# Create synthetic data for testing
def create_synthetic_data(num_samples=1000):
    """Create synthetic financial transaction data for testing."""
    np.random.seed(42)
    
    # Generate dates
    dates = pd.date_range(start='2023-01-01', periods=num_samples)
    
    # Generate amounts
    amounts = np.random.exponential(scale=100, size=num_samples)
    
    # Generate categories
    categories = np.random.choice(
        ['grocery', 'utilities', 'entertainment', 'travel', 'healthcare', 'other'],
        size=num_samples
    )
    
    # Generate description
    descriptions = [f"Transaction {i}" for i in range(num_samples)]
    
    # Generate clean descriptions
    clean_descriptions = [f"Clean {desc}" for desc in descriptions]
    
    # Generate fraud flag (rare event)
    is_fraud = np.random.choice([0, 1], size=num_samples, p=[0.98, 0.02])
    
    # Create DataFrame
    df = pd.DataFrame({
        'date': dates,
        'amount': amounts,
        'category': categories,
        'description': descriptions,
        'clean_description': clean_descriptions,
        'is_fraud': is_fraud
    })
    
    return df

# Custom server strategy that simply selects the best model
class BestModelStrategy(fl.server.strategy.Strategy):
    def __init__(self, min_fit_clients, min_evaluate_clients, min_available_clients):
        super().__init__()
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.current_parameters = None
        self.best_score = float("-inf")
        self.best_parameters = None
        
    def initialize_parameters(self, client_manager):
        """Initialize with empty parameters or from first client."""
        return self.current_parameters
    
    def configure_fit(self, server_round, parameters, client_manager):
        """Configure the next round of training."""
        # Sample clients
        sample_size = int(client_manager.num_available() * 1.0)
        sample_size = max(sample_size, self.min_fit_clients)
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=self.min_fit_clients)
        
        # Create and return config
        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in clients]
    
    def aggregate_fit(self, server_round, results, failures):
        """Select the model with the best training score."""
        if not results:
            return None, {}
        
        # Find the model with the best score
        best_round_score = float("-inf")
        best_round_parameters = None
        
        for _, fit_res in results:
            if "train_score" in fit_res.metrics and fit_res.metrics["train_score"] > best_round_score:
                best_round_score = fit_res.metrics["train_score"]
                best_round_parameters = fit_res.parameters
        
        # Update best overall model if this round's best is better
        if best_round_score > self.best_score:
            self.best_score = best_round_score
            self.best_parameters = best_round_parameters
        
        # Return the current round's best model
        self.current_parameters = best_round_parameters
        metrics = {"best_score": best_round_score}
        return best_round_parameters, metrics
    
    def configure_evaluate(self, server_round, parameters, client_manager):
        """Configure the next round of evaluation."""
        # Sample clients
        sample_size = int(client_manager.num_available() * 1.0)
        sample_size = max(sample_size, self.min_evaluate_clients)
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=self.min_evaluate_clients)
        
        # Create and return config
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)
        return [(client, evaluate_ins) for client in clients]
    
    def aggregate_evaluate(self, server_round, results, failures):
        """Aggregate evaluation results."""
        if not results:
            return None
        
        # Average metrics
        loss_aggregated = sum([evaluate_res.loss * evaluate_res.num_examples for _, evaluate_res in results]) / \
                         sum([evaluate_res.num_examples for _, evaluate_res in results])
        
        metrics_aggregated = {}
        for _, evaluate_res in results:
            for key, value in evaluate_res.metrics.items():
                if key not in metrics_aggregated:
                    metrics_aggregated[key] = []
                metrics_aggregated[key].append(value)
        
        for key in metrics_aggregated:
            metrics_aggregated[key] = sum(metrics_aggregated[key]) / len(metrics_aggregated[key])
        
        return loss_aggregated, metrics_aggregated
    
    def evaluate(self, server_round, parameters):
        """Skip centralized evaluation."""
        return None

# Flower client class
class FinancialModelClient(fl.client.NumPyClient):
    def __init__(self, client_id, model_type='fraud'):
        """Initialize the client."""
        self.client_id = client_id
        self.model_type = model_type
        
        # Try to load data or use synthetic data
        try:
            df, _ = load_and_preprocess_data('./transactions_data_extended.csv')
            print(f"Client {client_id}: Using real data")
        except FileNotFoundError:
            print(f"Client {client_id}: Using synthetic data")
            df = create_synthetic_data()
        
        # Partition the data for this client (simple strategy: each client gets all data)
        # In a real system, data would be naturally partitioned
        datasets = prepare_model_datasets(df)
        X, y = datasets[model_type]
        
        # Split data for this client
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X, y, test_size=0.2, random_state=42 + client_id
        )
    
        # Apply SMOTE for fraud detection
        if model_type == 'fraud':
            smote = SMOTE(random_state=42 + client_id)
            self.X_train, self.y_train = smote.fit_resample(self.X_train, self.y_train)
        
        # Create model based on type
        if model_type in ['fraud', 'budget']:
            # Compute scale_pos_weight for imbalanced fraud data
            if model_type == 'fraud':
                neg_count = sum(self.y_train == 0)
                pos_count = sum(self.y_train == 1)
                scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1
            else:
                scale_pos_weight = 1
            
            self.model = XGBClassifier(
                use_label_encoder=False,
                eval_metric='logloss',
                random_state=42 + client_id,
                max_depth=4,
                learning_rate=0.05,
                n_estimators=150,
                scale_pos_weight=scale_pos_weight
            )
            self.is_classifier = True
        else:
            self.model = XGBRegressor(
                random_state=42 + client_id,
                max_depth=4,
                learning_rate=0.05,
                n_estimators=150
            )
            self.is_classifier = False
        
        # Initial training for the local model
        self.train_local_model()
    
    def train_local_model(self):
        """Train the local model."""
        if self.is_classifier:
            self.model.fit(self.X_train, self.y_train)
        else:
            self.model.fit(self.X_train, self.y_train)
    
    def get_parameters(self, config):
        """Get model parameters."""
        # Serialize the model
        model_bytes = pickle.dumps(self.model)
        # Convert to numpy array
        return [np.frombuffer(model_bytes, dtype=np.uint8)]
    
    def set_parameters(self, parameters):
        """Set model parameters."""
        if parameters:
            # Convert numpy array back to bytes
            model_bytes = parameters[0].tobytes()
            # Deserialize
            try:
                self.model = pickle.loads(model_bytes)
            except Exception as e:
                print(f"Client {self.client_id}: Error deserializing model - {e}")
                # Keep using the current model
                pass
    
    def fit(self, parameters, config):
        """Train model and return parameters, number of examples, and metrics."""
        # Set parameters if provided
        if parameters and len(parameters) > 0:
            self.set_parameters(parameters)
        
        # Train model
        self.train_local_model()
        
        # Get updated parameters
        updated_parameters = self.get_parameters(config)
        
        # Calculate metrics
        if self.is_classifier:
            y_pred = self.model.predict(self.X_train)
            train_score = float(accuracy_score(self.y_train, y_pred))
        else:
            y_pred = self.model.predict(self.X_train)
            # For regression, higher is better, so use negative MSE
            train_score = -float(mean_squared_error(self.y_train, y_pred))
        
        # Return updated parameters and metrics
        print(f"Client {self.client_id}: Completed fit with score {train_score}")
        return updated_parameters, len(self.X_train), {"train_score": train_score}
    
    def evaluate(self, parameters, config):
        """Evaluate model on local data and return metrics."""
        # Set parameters if provided
        if parameters and len(parameters) > 0:
            self.set_parameters(parameters)
        
        # Evaluate model
        if self.is_classifier:
            y_pred = self.model.predict(self.X_val)
            loss = 1.0 - accuracy_score(self.y_val, y_pred)
            metrics = {
                "accuracy": float(accuracy_score(self.y_val, y_pred))
            }
        else:
            y_pred = self.model.predict(self.X_val)
            loss = float(mean_squared_error(self.y_val, y_pred))
            metrics = {
                "mse": loss
            }
        
        print(f"Client {self.client_id}: Evaluation - {metrics}")
        return loss, len(self.X_val), metrics

# Define client_fn function
def client_fn(cid):
    """Create a new client instance."""
    # Parse client ID
    client_id = int(cid)
    
    # Determine model type based on client ID
    model_types = ['fraud', 'budget', 'expense', 'forecast']
    model_type = model_types[client_id % len(model_types)]
    
    # Create and return client
    client = FinancialModelClient(client_id, model_type)
    return client

# Main federated learning simulation function
def main_fl():
    """Run federated learning simulation."""
    print("Starting federated learning simulation...")
    
    # Define strategy
    strategy = BestModelStrategy(
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2
    )
    
    # Configure server
    server = fl.server.Server(client_manager=fl.server.SimpleClientManager(), strategy=strategy)
    
    # Start simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=4,
        config=fl.server.ServerConfig(num_rounds=3),
        strategy=strategy,
        client_resources={"num_cpus": 1},  # Limit resources to prevent OOM errors
    )
    
    print("Federated learning simulation completed!")

# Run the simulation
if __name__ == "__main__":
    main_fl()

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout


Starting federated learning simulation...


2025-04-14 16:18:30,024	INFO worker.py:1852 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:172.19.2.2': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'memory': 21164322816.0, 'object_store_memory': 9070424064.0, 'GPU': 2.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 4 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=864) 2025-04-14 16:18:32.815667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=865) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=865) E0000 00:00:17446475

(ClientAppActor pid=863) Dataset shape: (1000, 18)
(ClientAppActor pid=863) Number of categories: 5
(ClientAppActor pid=863) Client 0: Using real data


(ClientAppActor pid=863) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=863) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=863) 
(ClientAppActor pid=863)             This is a deprecated feature. It will be removed
(ClientAppActor pid=863)             entirely in future versions of Flower.
(ClientAppActor pid=863)         
(ClientAppActor pid=866) 
(ClientAppActor pid=866)         
(ClientAppActor pid=865) 
(ClientAppActor pid=865)         
ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line

...

(ClientAppActor pid=863) Client 1: Completed fit with score 0.713520749665328 [repeated 3x across cluster]


ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 95, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 401, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 282, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
                                         ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/client_mode_hook.py", line

(ClientAppActor pid=863) Dataset shape: (1000, 18) [repeated 12x across cluster]
(ClientAppActor pid=863) Number of categories: 5 [repeated 12x across cluster]
(ClientAppActor pid=863) Client 0: Using real data [repeated 12x across cluster]
(ClientAppActor pid=863) Client 0: Evaluation - {'accuracy': 0.92}


(ClientAppActor pid=865) 
ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 95, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 401, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 282, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
                                         ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/

(ClientAppActor pid=865) Client 0: Evaluation - {'accuracy': 0.92}
Federated learning simulation completed!
